In [1]:
import gradio as gr
import joblib
import numpy as np

c:\Users\ENG_ABDELRAHMAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the trained Random Forest model and the scaler
model = joblib.load("random_forest_model.pkl")
scaler = joblib.load("scaler.pkl")

c:\Users\ENG_ABDELRAHMAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ENG_ABDELRAHMAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ENG_ABDELRAHMAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: Inconsis

In [3]:
# Define the prediction function that will be used by the Gradio interface

def predict(age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure,
            platelets, serum_creatinine, serum_sodium, smoking, time):
    
    try:
        #convet inputs to float
        age = float(age)
        anaemia = float(anaemia)
        cpk = float(creatinine_phosphokinase)
        diabetes = float(diabetes)
        ef = float(ejection_fraction)
        hbp = float(high_blood_pressure)
        platelets = float(platelets)
        creatinine = float(serum_creatinine)
        sodium = float(serum_sodium)
        smoking = float(smoking)
        time = float(time)

        # Create derived features (these were included in model training)

        platelets_per_age = platelets / (age + 1e-5)
        EF_per_age = ef / (age + 1e-5)
        time_ratio = time / (creatinine + 1e-5)

        # Combine all 14 features expected by the model
        features = np.array([[age, anaemia, cpk, diabetes, ef, hbp, platelets,
                              creatinine, sodium, smoking, time,
                              platelets_per_age, EF_per_age, time_ratio]])

        # Scale the features using the same scaler used in training
        features_scaled = scaler.transform(features)
                
        # Make prediction (1 = death, 0 = survived)
        prediction = model.predict(features_scaled)[0]

        # Return human-readable result
        return "❌ Death Expected" if prediction == 1 else "✅ Survived"

    except Exception as e:
        # Return any error that occurs (e.g., input format error)
        return f"⚠️ Error: {str(e)}"


In [4]:
# Define the Gradio interface for user input/output
iface = gr.Interface(
    fn=predict,   # Link to prediction function
    inputs=[
        gr.Number(label="Age"),
        gr.Radio([0, 1], label="Anaemia (1 = Yes, 0 = No)"),
        gr.Number(label="Creatinine Phosphokinase"),
        gr.Radio([0, 1], label="Diabetes (1 = Yes, 0 = No)"),
        gr.Number(label="Ejection Fraction"),
        gr.Radio([0, 1], label="High Blood Pressure (1 = Yes, 0 = No)"),
        gr.Number(label="Platelets"),
        gr.Number(label="Serum Creatinine"),
        gr.Number(label="Serum Sodium"),
        gr.Radio([0, 1], label="Smoking (1 = Yes, 0 = No)"),
        gr.Number(label="Follow-up Time (days)")
    ],
    outputs=gr.Text(label="Prediction Result"), # Display output as text
    title="Heart Failure Prediction ",  # App title
    
)

In [5]:
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
